In [4]:
# !pip install pyEDM 
# !pip install torchcde
# # !pip install gdown  # чтобы скачать данные с гугл диска

# from IPython.display import clear_output
# import numpy as np 
# import matplotlib.pyplot as plt
# import seaborn as sns
# from scipy.stats import pearsonr
# import numpy.random as npr
# import pandas as pd
# import pyEDM   # реализация CCM
# import torchdiffeq
# import os
# import sys
# from importlib import reload 
# from google.colab import drive
# import torch

drive.mount('')
# подключение к папкам
os.chdir("drive/MyDrive/Colab Notebooks/m1p")
sys.path.insert(0, os.getcwd())

clear_output()

ValueError: ignored

In [3]:
ls 

sample_data/


# Новый раздел

In [1]:
from numpy.lib.shape_base import column_stack
import code_folder.accelerometer_gyroscope_data_prepare as data
import code_folder.train as train
import code_folder.ODE_on_RNN as ode_on_rnn
import code_folder.ODE_RNN as ode_rnn

def train_axis(axis, train_data, test_data, net, epochs = 20, optim = None):
    def epoch_dataset(dataset, epochs):
        for i in range(epochs):
            yield dataset
    train_dataset = epoch_dataset(train_data, epochs)
    test_dataset  = epoch_dataset(test_data, epochs)

    net = train.train_with_epoch(net, train_data, test_data, epochs, optim = optim)
    return net

def get_hidden(net, dataset):
    # print(train_data[0][0].shape)
    h_s = []
    net = net.cpu()
    for X, y , t in dataset:
        X = X.float()
        t = t.float()
        val, hid, _ = net(X, t, return_hidden = True)
        h_s.append(hid[-1])  # скрытое состояние последнего в эмбеддинге
    h_s = np.array(torch.hstack(h_s))
    return h_s

def get_hidden_long(net, dataset):
    # print(train_data[0][0].shape)
    h_s = []
    net = net.cpu()
    for X, y , t in dataset:
        X = X.float()
        t = t.float()
        val, hid, _ = net(X, t, return_hidden = True)
        h_s.append(hid)  # скрытое состояние последнего в эмбеддинге
    h_s = np.array(torch.hstack(h_s))
    return h_s

def apply_CCM(d_s_1, d_s_2, time, to, d_s_1_columns, d_s_2_columns, libSizes = None):
    if libSizes is None: libSizes = "10 120 5"
    d_s_1 = d_s_1[:to]
    d_s_2 = d_s_2[:to]
    time = time[:to]
    data_ = pd.DataFrame(np.hstack([time.squeeze(), d_s_1[:-1], d_s_2[:-1]]),
            columns = ['time', *d_s_1_columns, *d_s_2_columns])
    tmp = pyEDM.CCM(dataFrame = data_, 
                    columns = d_s_1_columns, target = d_s_2_columns,
                    embedded = True, libSizes= libSizes, 
                    sample = 10, showPlot = True)

def apply_CCM_on_dataset(dataset, to, d_s_1_columns, d_s_2_columns, libSizes = None):
    if libSizes is None: libSizes = "10 120 5"
    data_ =dataset.iloc[:to]
    tmp = pyEDM.CCM(dataFrame = data_, 
                    columns = d_s_1_columns, target = d_s_2_columns,
                    embedded = True, libSizes= libSizes, 
                    sample = 10, showPlot = True)
    
embedding_dim = 10
filenames = ["./unzipped/5/Accelerometer.csv","./unzipped/5/Gyroscope.csv"]
gyroscope_axis = ["G:x", "G:y", "G:z"]
gyroscope_embedding_dim = 10

accelerometer_axis = ["A:x", "A:y", "A:z"]
accelerometer_embedding_dim = 10

gyroscope_data_dim = 3
gyroscope_embedding_dim = 10
len_ = 2000
all_data = data.DataPreprocess(*filenames).preprocess(torch.embedding_dim, gyroscope_axis, + accelerometer_axis)
accelerometer_train, accelerometer_test = data.make_datasets(accelerometer_axis, *filenames, accelerometer_embedding_dim)
gyroscope_train, gyroscope_test         = data.make_datasets(gyroscope_axis, *filenames, gyroscope_embedding_dim)
random_train, random_test               = data.make_random_dataset(gyroscope_data_dim, gyroscope_embedding_dim, len_)

ModuleNotFoundError: ignored

Просто CCM